In [1]:
from collections import defaultdict

import numpy as np
import pandas as pd
import warnings

from surprise import Dataset, Reader
from tqdm import tqdm

from src.models import cf

tqdm.pandas()
warnings.filterwarnings('ignore')

/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Load Data

In [2]:
# global variables
DATA_PATH = "data/evaluation"
CATEGORY = "Pet_Supplies"

# training parameters
N_EPOCHS = 10
LR_ALL = 0.005
BETA = 0.1

train = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_train.csv")

In [3]:
# checking train dataframe
train.head().append(train.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,0,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A14CK12J7C7JRK,3.0,I purchased the Trilogy with hoping my two cat...,2011-01-12,purchase trilogy hop cat age interested yr old...
1,2,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A2CR37UY3VR7BN,4.0,I bought the triliogy and have tested out all ...,2012-12-19,buy triliogy test dvd appear volume receive re...
2,3,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A2A4COGL9VW2HY,4.0,My female kitty could care less about these vi...,2011-05-12,female kitty care video care little male dig a...
3,4,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A2UBQA85NIGLHA,3.0,"If I had gotten just volume two, I would have ...",2012-03-05,volume star trilogy star read review know vol ...
4,5,B00005MF9U,LitterMaid LM900 Mega Self-Cleaning Litter Box,"['Pet Supplies', 'Cats', 'Litter &amp; Housebr...",A2BH04B9G9LOYA,1.0,"First off, it seems that someone is spamming t...",2006-12-31,spamming review glow reviewer review amazon ba...
68865,111581,B00K3YPOO0,Brightest Black Light Flashlight on Amazon- UV...,[],A11J1FHCK5U06J,4.0,Now I know exactly where the trouble spots are...,2014-05-23,know exactly trouble spot sniffing guess invis...
68866,111585,B00K3YPOO0,Brightest Black Light Flashlight on Amazon- UV...,[],A18JF0T0GOCORW,4.0,I use this light to help me find stains when I...,2014-05-24,use light help stain carpet clean pre treat ca...
68867,111595,B00K7EG97C,Nutro Crunchy Dog Treats with Real Mixed Berri...,"['Pet Supplies', 'Dogs', 'Treats', 'Cookies, B...",A3GRPCW9DG427Z,5.0,We are owned by the 3 pickiest pooches in the ...,2013-07-27,pickiest pooch world love fool reject doggie t...
68868,111598,B00K7EG97C,Nutro Crunchy Dog Treats with Real Mixed Berri...,"['Pet Supplies', 'Dogs', 'Treats', 'Cookies, B...",A2X6TLAX3JEO1A,5.0,My highly allergic white boxer loves these tre...,2014-05-09,highly allergic white boxer love treat meat co...
68869,111602,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A9PG9ODPPP31N,5.0,Works great on my medium sized dog. She has ve...,2014-07-09,work great medium size dog coarse hair work gr...


# Preparing Dataset for Surprise's Algorithm

In [4]:
# create reader 
reader = Reader(rating_scale=(1,5))

# Utility Functions

In [5]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in tqdm(predictions):
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in tqdm(top_n.items()):
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def recall_at_k(asins, predicted_asins, k=10):
    # number of relevant items
    set_actual = set(asins)
    set_preds = set(predicted_asins)
    num_relevant = len(set_actual.intersection(set_preds))
    
    # calculating recall@K - relevant / total relevant items
    recall_at_k = num_relevant / len(asins)
    
    return recall_at_k

def novelty_at_k(item_popularity, predicted_asins, k=10):
    """
    """
    # finding avg novelty
    popularity_sum = item_popularity.loc[predicted_asins].sum()
    novelty_at_k = ((k*1) - popularity_sum) / k
    
    return novelty_at_k

def generate_item_popularity(train: pd.DataFrame) -> pd.DataFrame:
    """
    """
    
    # create a mapping of item popularatity
    # based on sum(item's review / max reviews) / no items
    max_reviews = (train.groupby(['asin'])
                   .agg({'processedReviewText': 'count'})
                   .max()
                   .values[0])
    item_popularity = (train.groupby(['asin'])
                       .agg({'processedReviewText': 'count'})
                       .apply(lambda x: x/max_reviews))
    
    return item_popularity
    

def evaluate_recommendations(top_ns: dict, user_rating_history: pd.DataFrame, item_popularity: pd.DataFrame, k=10) -> pd.DataFrame:
    """
    
    Args:
        top_ns
        user_rating_history
    """
    
    test_recommendations = pd.DataFrame(top_ns.items(), columns=["reviewerID", "pred_asin"])
    test_recommendations['pred_asin'] = test_recommendations['pred_asin'].apply(lambda x: [i[0] for i in x])
    
    # combined test history and recommendations
    test_merged = pd.merge(user_rating_history, test_recommendations, on="reviewerID", how="inner")
    
    # generating recall@k metrics
    test_merged["recall@k"] = test_merged.apply(lambda x: recall_at_k(x.asin, x.pred_asin, k=k), axis=1)
    test_merged["novelty@k"] = test_merged.apply(lambda x: novelty_at_k(item_popularity, x.pred_asin, k=k), axis=1)
    average_recall_at_k = test_merged["recall@k"].mean()
    average_novelty_at_k = test_merged["novelty@k"].mean()
    
    print(f"The MEM-ECF has an average recall@{k}: {average_recall_at_k:.5f}, average novelty@{k}: {average_novelty_at_k:.5f}")
    
    return test_merged

# Generate N-Recommendations = {10, 25, 30, 45}

## Load Test Data

In [6]:
test = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_test.csv")

In [7]:
test.head().append(test.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,1,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A39QHP5WLON5HV,5.0,There are usually one or more of my cats watch...,2013-09-14,usually cat watch tv stay trouble dvd play lik...
1,104,B00005MF9V,LitterMaid Universal Cat Privacy Tent (LMT100),"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",A366V0GCEPH5CX,5.0,My cats love it and so do I. I no longer have ...,2013-02-02,cat love longer cat litter fly floor litter fl...
2,133,B00005MF9T,LitterMaid LM500 Automated Litter Box,"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",ALWWS8QBYN80B,1.0,I have one female cat that weighs under 10 pou...,2004-11-17,female cat weigh pound year old use everclean ...
3,153,B00005MF9W,LitterMaid Waste Receptacles Automatic Litter ...,"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",A3PVI3NE7OY1SP,5.0,I love these. They make the clean up so much e...,2013-09-26,love clean easy clean box manually use issue w...
4,154,B00005MF9W,LitterMaid Waste Receptacles Automatic Litter ...,"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",A2H83XMHUVDLJY,4.0,"I love this litter box. I do not use the lids,...",2014-06-26,love litter box use lid use receptacle tear cr...
41564,111601,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],AV34KNYW82YSS,4.0,Pulled lots of hair out of my Labs coat. Didn'...,2014-07-18,pulled lot hair labs coat think prove wrong co...
41565,111603,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A1YMNTFLNDYQ1F,5.0,I have been trying to find a rubber bristle br...,2014-07-16,try rubber bristle brush persian year lose glo...
41566,111604,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A1FQ3HRVXA4A5B,5.0,Great product to use on your pets knowing this...,2014-07-11,great product use pet know gentle rubber damag...
41567,111605,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A3OP6CI0XCRQXO,5.0,I bought a second one because I have two cats ...,2014-07-22,buy second cat american short hair buy brush m...
41568,111606,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A11LC938XF35XN,5.0,Our dogs love getting brushed with this. It m...,2014-07-17,dog love brush massage remove heavy undercoat ...


In [8]:
# generating test history
test_user_history = (pd.DataFrame(test.groupby(['reviewerID'])['asin']
                                  .apply(list).reset_index()))

In [9]:
print(test_user_history)

                  reviewerID                                  asin
0      A04173782GDZSQ91AJ7OD              [B0090Z9AYS, B00CPDWT2M]
1      A042274212BJJVOBS4Q85              [B005AZ4M3Q, B00771WQIY]
2       A0436342QLT4257JODYJ  [B0018CDR68, B003SJTM8Q, B00474A3DY]
3      A04795073FIBKY8GSLZYI              [B001PKT30M, B005DGI2RY]
4      A06658082A27F4VB5UG8E              [B000TZ1TTM, B0019VUHH0]
...                      ...                                   ...
18993          AZYJE40XW6MFG              [B00HVAKJZS, B00IDZT294]
18994          AZZ56WF4X19G2                          [B004A7X218]
18995          AZZNK89PXD006  [B0002DHV16, B005BP8MQ8, B009RTX4SU]
18996          AZZV9PDNMCOZW              [B007EQL390, B00ISBWVT6]
18997          AZZYW4YOE1B6E  [B0002AQPA2, B0002AQPA2, B0002ARQV4]

[18998 rows x 2 columns]


# Instantiate FunkSVD (Matrix Factorization)

In [10]:
# instantiating funksvd 
funk_svd = cf.FunkMF(n_epochs=N_EPOCHS, lr_all=LR_ALL, reg_all=BETA)

In [11]:
# fitting to the training data
funk_svd.fit(train)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [12]:
%%time
# creating testset for prediction
testset = funk_svd.trainset.build_anti_testset()

CPU times: user 46.9 s, sys: 2.09 s, total: 48.9 s
Wall time: 49.1 s


In [13]:
%%time
# generate candidate times
candidate_items = funk_svd.test(testset)

CPU times: user 10min 30s, sys: 5min 16s, total: 15min 47s
Wall time: 17min 52s


## Loop through N = {10, 25, 30, 45}

In [14]:
# generate item popularity
item_popularity = generate_item_popularity(train)

In [15]:
n_recommendations = {}
for n in [10, 25, 30, 45]:
    # retrieve the top-n items based on similarities
    top_ns = get_top_n(candidate_items, n)
    # evaluate how well the recommended items predicted the future purchases
    n_recommended_items = evaluate_recommendations(top_ns, test_user_history, item_popularity, n)
    # saving the n-value and recommended items
    n_recommendations[n] = (top_ns, n_recommended_items)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19058/19058 [01:04<00:00, 296.84it/s]


The MEM-ECF has an average recall@10: 0.00752, average novelty@10: 0.88427


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19058/19058 [01:04<00:00, 295.70it/s]


The MEM-ECF has an average recall@25: 0.01510, average novelty@25: 0.88741


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19058/19058 [01:10<00:00, 271.30it/s]


The MEM-ECF has an average recall@30: 0.01753, average novelty@30: 0.89002


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19058/19058 [01:05<00:00, 289.14it/s]


The MEM-ECF has an average recall@45: 0.02401, average novelty@45: 0.89838


# Evaluate N-Recommendations

In [16]:
def retrieve_recommendations(train: pd.DataFrame, top_ns: dict):
    """
    """
    # generating a random user
    random_user = np.random.choice(list(train['reviewerID'].unique()), 1)[0]
    print(f"For user: {random_user}:")
    print(f"Purchase History:\n{train[train['reviewerID'] == random_user][['asin', 'title']]}")

    # find the recommendations
    print(f"\nRecommending:\n")
    recommendations = (train[train['asin']
                             .isin([i[0] for i in top_ns[random_user]])][['asin', 'title']]
                       .drop_duplicates(subset='asin')
                       .set_index('asin'))
    print(f"{recommendations.loc[[i[0] for i in top_ns[random_user]]].reset_index()}")

## N=10

In [17]:
top_ns_10 = n_recommendations[10][0]
retrieve_recommendations(train, top_ns_10)

For user: A2QJHUMBOYTG39:
Purchase History:
             asin                                              title
14820  B0002ASCGM  Van Ness Odor Control Extra Giant Enclosed Cat...
30249  B0009YF3KY              Petstages Dental Health Cat Chew Toys
45219  B001709GF2           Merrick Before Grain Meat-11.1-Pound Bag
64104  B005OB3E30  Purina Fancy Feast Gravy Lovers Poultry &amp; ...

Recommending:

         asin                                              title
0  B001LNSSH2  ZYMOX Pet King Brand Otic Pet Ear Treatment wi...
1  B0009YUIZE  Petmate FATCAT Big Mama's Scratchy Box Cardboa...
2  B0017JE6B2                                 Pet Food Container
3  B0002563O0                           JW Comfy Perch for Birds
4  B000F4AVPA                                Chuckit! Ultra Ball
5  B000255OIG       Stewart Freeze Dried Treats 14 oz Beef Liver
6  B00008DFOG                     Redbarn Pet Products Food Roll
7  B0002DJONY                            Vittles Vault Stackable
8  B0002AS

## N=25

In [18]:
top_ns_25 = n_recommendations[25][0]
retrieve_recommendations(train, top_ns_25)

For user: A34PPIW1NC9LK2:
Purchase History:
             asin                                              title
1522   B000084E6V                      Nylabone Dental Dinosaur Chew
63160  B005AP3B8S                        KONG Cozies Dog Squeaky Toy
63347  B005CQARAA  Outward Hound Stuffingless Gecko Toss and Tug ...

Recommending:

          asin                                              title
0   B0002AQKIO        Fluval Carbon, 100-gram Nylon Bags - 3-Pack
1   B003JFRQQ4  Scaredy Cut Tiny Trim by Small Pet Grooming Sa...
2   B00025Z6YI  TetraFin Balanced Diet Goldfish Flake Food for...
3   B0002DJONY                            Vittles Vault Stackable
4   B0017JE6B2                                 Pet Food Container
5   B000A6UF4U     Gamma2 Vittles Vault Plus for Pet Food Storage
6   B001LNSSH2  ZYMOX Pet King Brand Otic Pet Ear Treatment wi...
7   B000F4AVPA                                Chuckit! Ultra Ball
8   B000255MZG         API STRESS COAT Aquarium Water Conditioner
9   

## N=30

In [19]:
top_ns_30 = n_recommendations[30][0]
retrieve_recommendations(train, top_ns_30)

For user: A1IKNQ7YYGGXNJ:
Purchase History:
             asin                                              title
28671  B0006TQU36  PetSafe SlimCat Interactive Toy and Food Dispe...
38344  B000LPOUNW             Go Cat Teaser Cat Catcher Wand Cat Toy

Recommending:

          asin                                              title
0   B003BYQ1C8                            Armarkat Cat Tree Model
1   B00025Z6YI  TetraFin Balanced Diet Goldfish Flake Food for...
2   B0017JE6B2                                 Pet Food Container
3   B000296N7S  Panacur C Canine Dewormer, Net Wt. 12 Grams, P...
4   B0002563O0                           JW Comfy Perch for Birds
5   B0002DJONY                            Vittles Vault Stackable
6   B000A6UF4U     Gamma2 Vittles Vault Plus for Pet Food Storage
7   B000K9JRH8  GoCat DaBird Feather Refill, Assorted Colors, ...
8   B000FWAP8A  GoCat Da Bird Rod and Feather Cat Toy, Handmad...
9   B000F4AVPA                                Chuckit! Ultra Ball
10  B00

## N=45

In [20]:
top_ns_45 = n_recommendations[45][0]
retrieve_recommendations(train, top_ns_45)

For user: A3IHROYFSVJUMK:
Purchase History:
             asin                       title
30144  B0009YD8NS  Treat Dispensing Chew Ball

Recommending:

          asin                                              title
0   B001LNSSH2  ZYMOX Pet King Brand Otic Pet Ear Treatment wi...
1   B0009YUIZE  Petmate FATCAT Big Mama's Scratchy Box Cardboa...
2   B000K9JRH8  GoCat DaBird Feather Refill, Assorted Colors, ...
3   B0002563O0                           JW Comfy Perch for Birds
4   B003JFRQQ4  Scaredy Cut Tiny Trim by Small Pet Grooming Sa...
5   B0002DJONY                            Vittles Vault Stackable
6   B000296N7S  Panacur C Canine Dewormer, Net Wt. 12 Grams, P...
7   B00025Z6Q6  Tetra TetraCichlid Balanced Diet Flakes Food f...
8   B000255NCI                               API Master Test Kits
9   B000F4AVPA                                Chuckit! Ultra Ball
10  B0002AS1CC                 Bergan Turbo Scratcher Accessories
11  B000MLG4K2  Wellness Wellbites Soft Natural Dog Trea